In [1]:
import pandas as pd
import numpy as np
import requests
import re
from collections import namedtuple
from datetime import date, datetime


In [31]:
#reading in data
url = 'https://docs.google.com/spreadsheets/d/1iBv2cj6u1MH_8JSvEfN-aJpTveyhkkgsNrLrgj1I3_U/edit#gid=1098993272'
url_parts = url.split("#")    #spliting url by #
url = url_parts[0].replace("edit", "export?") + url_parts[1] + "&format=csv" # parsing url 
data = pd.read_csv(url,header = 0 )   #reading in data
data.fillna('', inplace = True)   # filling missing data
data.drop('comment', axis = 1)    # droping the COMMENT column
names = [x.strip() for x in data.Name.values]
if 'Abdalla' in names:
    data.iloc[0,:]['busy_day'] = 'mon-fri' # using location to impute row one | need to be changed



In [33]:
data.Name.nunique()

15

In [174]:
year = datetime.now().year
mpat = r'[\d]+'
monthpat = r'[A-Za-z]+'
mpat = re.compile(mpat)
monthpat = re.compile(monthpat)


months_long = ["january","february", 'march','march','april','april','may','june','july', 'august','september','october'\
          ,'november','december']

months_short = ["jan","feb", 'mar','march', 'april','april','may','june','july', 'aug','sept','oct'\
          ,'nov','dec']

short_to_long = dict(zip(months_short,months_long))


months_to_num = dict(zip(months,range(1,13)))
num_to_months = dict(zip(range(1,13),months))

days_map={
    "mon":"monday",
    "tues": "tuesday",
    "tue" : 'tuesday',
    "wed": "wednesday",
    "thurs": "thursday",
    "thur" : 'thursday',
    "fri": "friday",
    "sat": "saturday",
    "sun": "sunday"
}

day_to_num= {
    "monday":0,
    "tuesday":1,
    "wednesday": 2,
    "thursday": 3,
    "friday": 4,
    "saturday": 5,
    "sunday": 6
}

num_to_day = {
    0:  "monday",
    1: "tuesday",
    2:  "wednesday",
    3:  "thursday",
    4: "friday",
    5: "saturday",
    6: "sunday"
}
#-------------------------------------------------------------         

def parse_days(days):
    """Takes a list of days and parse it to a long version
    e.g :
        mon = monday
        tues = tuesday
        
        return parse list of days
    """
    assert type(days) == list, 'days must be a list'
    days = [day.lower() for day in days]
    
    length_one = len(days[0])
    length = len(days)
    
    if length == 1 and 3<= length_one <=5:
        return [days_map[days[0]]]
    
    elif length == 1 and length_one> 5:
        return days
    
    else:
        for i,day in enumerate(days):
            day = day.strip()
            if 3<=len(day)<=5:
                days[i] = days_map[day]
            else:
                pass
        return days
    
def parse_month(month):
    """Converts months to integers
    """
    month = month.lower()
    length = len(month)
    if 3<= length <=5 :
        return months_to_num[short_to_long[month]]
    return months_to_num[month]
        
    
def expand_day(From='monday', To='sunday'):
    """Takes to separate days of the week and expands it
    e.g: 
    ['monday', 'tuesday',...]
    """
    
    From = day_to_num[From.lower()]
    To = day_to_num[To.lower()]
    return [num_to_day[i] for i in range(From, To+1)]

#named tuple object -----------------------------------------
Observation = namedtuple("Observation", 'Name From To Busy_day')
# Processing of observation --------------------------------------

def obs_generator(df):
    """Takes data and reads each record"""
    
    n = df.shape[0]
    try:
        ndata = df.drop('comment', axis = 1)
    except ValueError:
        ndata = df

    for i in range(n):
        row = ndata.iloc[i,].values
        yield Observation(row[0], row[1], row[2],row[3])
    
def parse_observation(obs = ''):
    

    
    # try to extract month  and day from string
    try:
        mday_from = mpat.search(obs.From).group()
        month_from = monthpat.search(obs.From).group()
        mday_to = mpat.search(obs.To).group()
        month_to = monthpat.search(obs.To).group()
    except AttributeError:
        mday_from = ''
        month_from = ""
        mday_to = ''
        month_to = ''
    #creating a list of days
    if ',' in obs.Busy_day and 'and' not in obs.Busy_day:
        busy_day = obs.Busy_day.split(',')
        busy_day = parse_days(busy_day)
        
    elif '-' in obs.Busy_day:
        busy_day = obs.Busy_day.split('-')
        busy_day = parse_days(busy_day)
        busy_day = expand_day(busy_day[0],busy_day[1])
        
    elif "and" in obs.Busy_day and ',' in obs.Busy_day:
        out = []
        busy_day = obs.Busy_day.split(",")
        busy_day = [x.split('and') for x in busy_day]
        for ls in busy_day:
            out.extend(ls)
        busy_day = out
        busy_day = parse_days(busy_day)
        
    else: 
        busy_day = parse_days([obs.Busy_day]) 
    
    #parsing the dates to datetime objects
    Name = obs.Name
    if mday_from != '':
        From = date(year,parse_month(month_from),int(mday_from))
        To = date(year, parse_month(month_to), int(mday_to))
    else:
        From = ''
        To = ''
    
    return Observation(Name, From, To, busy_day)
    

def create_record():
    out = {}
    for observation in obs_generator(data):
        observation = parse_observation(observation)
        if observation.From != '':
            absent_days = [date.date() for date in pd.date_range(observation.From, end = observation.To, freq= 'D')]
        else:
            absent_days = []
        busy_days = [day for day in observation.Busy_day if day != '']
        if observation.Name in out:
            out[observation.Name.strip()]['away'].extend(absent_days)
            out[observation.Name.strip()]['preferred'].extend(busy_days)
        else:
            out[observation.Name.strip()]= {'away': absent_days, 'preferred': busy_days}
    return out

    

create_record()["Muyiwa"]

{'away': [datetime.date(2017, 2, 9),
  datetime.date(2017, 2, 10),
  datetime.date(2017, 2, 11),
  datetime.date(2017, 2, 16),
  datetime.date(2017, 2, 17),
  datetime.date(2017, 2, 18),
  datetime.date(2017, 2, 19),
  datetime.date(2017, 2, 23),
  datetime.date(2017, 2, 24),
  datetime.date(2017, 2, 25),
  datetime.date(2017, 2, 26),
  datetime.date(2017, 3, 9),
  datetime.date(2017, 3, 10),
  datetime.date(2017, 3, 11),
  datetime.date(2017, 3, 12),
  datetime.date(2017, 3, 13),
  datetime.date(2017, 3, 14),
  datetime.date(2017, 3, 15),
  datetime.date(2017, 3, 16),
  datetime.date(2017, 3, 17),
  datetime.date(2017, 3, 18),
  datetime.date(2017, 3, 19),
  datetime.date(2017, 4, 27),
  datetime.date(2017, 4, 28),
  datetime.date(2017, 4, 29),
  datetime.date(2017, 4, 30),
  datetime.date(2017, 5, 20),
  datetime.date(2017, 5, 21)],
 'preferred': ['monday', 'wednesday', 'thursday']}

In [150]:
def parse_from(x):
    try:
        mday_from = mpat.search(x).group()
        month_from = monthpat.search(x).group()
    except AttributeError:
        mday_from = ''
        month_from = ""
     
    if mday_from != '':
        From = date(year,parse_month(month_from),int(mday_from))

    else:
        From = ''
    return From

def parse_to(x):
    try:
        mday_to = mpat.search(x).group()
        month_to = monthpat.search(x).group()
    except AttributeError:
        mday_to = ''
        month_to = ''
     
    if mday_to != '':
        To = date(year, parse_month(month_to), int(mday_to))
    else:
        To = ''
    return To

data['date_from']=data.From.apply(parse_from)
data['date_to'] = data.To.apply(parse_to)
ndata = data.fillna('', axis = 1)

In [63]:
kh = pd.read_csv('/media/aisik/aisik/Documents/Steiner_schedules/2017-01-18TKH.csv', index_col = [0])
cook = pd.read_csv('/media/aisik/aisik/Documents/Steiner_schedules/2017-01-18Tcooking.csv',index_col = [0])
df = pd.merge(cook,kh, how = 'inner', left_on  = 'Date', right_on = "Date")
df = df.drop(['Day_y'],axis = 1)
df.sort_values("Date", inplace = True)
df.columns= ['Date','Day', 'Cook','Kitchen Helper']
df.to_csv('/media/aisik/aisik/Documents/Steiner_schedules/2017-01-18Tcooking.csv')

,Date,Day,Cook,Kitchen Helper
13,2017-01-21,saturday,Haruna,Essber
10,2017-01-22,sunday,Tuka,Tenzing
12,2017-01-23,monday,Sarah,Isaac
2,2017-01-24,tuesday,Moha,Henri
14,2017-01-25,wednesday,Henri,Moha
11,2017-01-26,thursday,Sherise,Sarah
5,2017-01-27,friday,Isaac,Rohit
3,2017-01-28,saturday,Abdalla,Amy
9,2017-01-29,sunday,BT,Haruna
7,2017-01-30,monday,Rohit,George
